# 第4章 word2vec的高速化

随着语料库中处理的词汇量的增加，计算量也随之增加。
当词汇量达到一定程度之后，上一章的 CBOW 模型的计算就会花费过多的时间。

简单的 word2vec 进行两点改进：引入Embedding层和Negative Sampling的新的损失函数。


## 4.1 word2vec的改进1

![](../images/图4-1.上一章中实现的CBOW模型.PNG)
图4-1.上一章中实现的CBOW模型

![](../images/图4-2.假设词汇量为100万个时的CBOW模型.PNG)
图4-2.假设词汇量为100万个时的CBOW模型

以下两个地方的计算会出现瓶颈。
* 输入层的 one-hot 表示和权重矩阵 $W_in$ 的乘积（4.1 节解决，引入Embedding层解决）
* 中间层和权重矩阵 $W_out$ 的乘积以及 Softmax 层的计算（4.2 节解决，引入Negative Sampling新损失函数）

### 4.1.1 Embedding层

![](../images/图4-3.one-hot表示的上下文和MatMul层的权重的乘积.PNG)
图4-3.one-hot表示的上下文和MatMul层的权重的乘积

创建一个从权重参数中抽取“单词 ID 对应行（向量）”的层，这里我们称之为 Embedding 层。
顺便说一句，Embedding 来自“词嵌入”（word embedding）这一术语。也就是说，
在这个 Embedding 层存放词嵌入（分布式表示）。

Embedding层：从权重参数中抽取“单词ID对应行（向量）”的层。
Embedding来自“词嵌入”（word embedding）这一术语，在Embedding层放入词嵌入（分布式表示）。

在自然语言处理领域，单词的密集向量表示称为**词嵌入**（word embedding）或者单词的**分布式表示**（distributed representation）。

### 4.1.2 Embedding层的实现

In [1]:
import numpy as np

W = np.arange(21).reshape(7, 3)
W

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [2]:
W[2]  # 从这个权重中取出某个特定的行

array([6, 7, 8])

In [3]:
idx = np.array([1, 0, 3, 0])
W[idx]  # 从这个权重中取出多行

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

Embedding 层实现如下所示（ [common/layers.py](../common/layers.py)）

In [ ]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None

    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out

Embedding 层的正向传播只是从权重矩阵 $W$ 中提取特定的行，并将该特定行的神经元原样传给下一层。
因此，在反向传播时，从上一层（输出侧的层）传过来的梯度将原样传给下一层（输入侧的层）。
不过，从上一层传来的梯度会被应用到权重梯度 dW 的特定行（idx）。

![](../images/图4-4.Embedding层的正向传播和反向传播处理的概要（Embedding层记为Embed）.PNG)
图4-4.Embedding层的正向传播和反向传播处理的概要（Embedding层记为Embed）

将 word2vec（CBOW 模型）的实现中的输入侧的 MatMul 层换成 Embedding 层。这样
一来，既能减少内存使用量，又能避免不必要的计算。

![](../images/图4-5.当idx数组的元素中出现相同的行号时，简单地将dh的行写入对应位置就会有问题.PNG)
图4-5.当idx数组的元素中出现相同的行号时，简单地将dh的行写入对应位置就会有问题

## 4.2 word2vec的改进2

使用Negative Simpling（**负采样**）替代Softmax，无论词汇量有多大，都可以使计算量保持较低或恒定。

### 4.2.1 中间层之后的计算问题

![](../images/图4-6.词汇量为100万个时的word2vec：上下文是you和goodbye，目标词是say.PNG)
图4-6　词汇量为100万个时的word2vec：上下文是you和goodbye，目标词是say

下两个地方需要很多计算时间。
* 中间层的神经元和权重矩阵（$W_{out}$）的乘积
* Softmax 层的计算

很有必要将矩阵乘积计算“轻量化”。

### 4.2.2 从多分类到二分类

用二分类（binary classification）拟合多分类（multiclass classification），这是理解负采样的重点。

![](../images/图4-7.仅计算目标词的得分的神经网络.PNG)
图4-7.仅计算目标词的得分的神经网络

![](../images/图4-8.计算say对应的列向量和中间层的内积（图中的“dot”指内积运算）.PNG)
图4-8.计算say对应的列向量和中间层的内积（图中的“dot”指内积运算）


### 4.2.3 sigmoid函数和交叉熵误差

sigmoid函数：
$$
    y = \frac{1}{1 + \exp(-x)} \tag{4.2}
$$

![](../images/图4-9.Sigmoid层（左图）和sigmoid函数（右图）的图像.PNG)
图4-9.Sigmoid层（左图）和sigmoid函数（右图）的图像


$$
    L = -\sum_{k}t_k\log y_k \tag{1.7}
$$

用于 sigmoid 函数的损失函数
$$
    L = -(t\log y+(1-t)\log(1-y))    \tag{4.3}
$$
y 是 sigmoid 函数的输出，t 是正确解标签，取值为 0 或 1


![](../images/图4-10.Sigmoid%20层和%20Cross%20Entropy%20Error%20层的计算图。右图整合为了%20Sigmoid.PNG)
图4-10.Sigmoid 层和 Cross Entropy Error 层的计算图。右图整合为了 Sigmoid with Loss层

### 4.2.4 从多分类到二分类的实现

![](../images/图4-11.进行多分类的CBOW模型的全貌图（Embedding层记为Embed）.PNG)
图4-11.进行多分类的CBOW模型的全貌图（Embedding层记为Embed）

![](../images/图4-12.进行二分类的CBOW模型的全貌图.PNG)
图4-12.进行二分类的CBOW模型的全貌图

将中间层的神经元记为 $h$


引入 Embedding Dot 层，该层将图 4-12 中的 Embedding 层和 dot运算（内积）合并起来处理。
使用这个层，图 4-12 的后半部分可以画成图 4-13。

![](../images/图4-13.只关注图4-12的中间层之后的处理。使用Embedding%20Dot层合并Embedding.PNG)
图4-13.只关注图4-12的中间层之后的处理。使用Embedding Dot层合并Embedding.PNG

Embedding Dot 层的实现，这里我们将这个层实现为 EmbeddingDot 类（ [ch04/negative_sampling_layer.py](../ch04/negative_sampling_layer.py)）。

![](../images/图4-14.Embedding%20Dot层中各个变量的值.PNG)
图4-14.Embedding Dot层中各个变量的值

### 4.2.5 负采样

![](../images/图4-15.CBOW模型的中间层之后的处理：上下文是you和goodbye，此时目标词是say的概率为0.993（99.3%25）.PNG)
图4-15.CBOW模型的中间层之后的处理：上下文是you和goodbye，此时目标词是say的概率为0.993（99.3%）

![](../images/图4-16.如果say是正例（正确答案），则当输入say时使Sigmoid层的输出接近1，当输入say以外的单词时使输出接近0，我们要求的是这样的权重.PNG)
图4-16.如果say是正例（正确答案），则当输入say时使Sigmoid层的输出接近1，当输入say以外的单词时使输出接近0，我们要求的是这样的权重


只使用少数负例。这就是负采样方法的含义。

负采样方法既可以求将正例作为目标词时的损失，同时也可
以采样（选出）若干个负例，对这些负例求损失。然后，将这些数据（正例
和采样出来的负例）的损失加起来，将其结果作为最终的损失。


![](../images/图4-17.负采样的例子（只关注中间层之后的处理，画出基于层的计算图）.PNG)
图4-17.负采样的例子（只关注中间层之后的处理，画出基于层的计算图）

### 4.2.6 负采样的采样方法

![](../images/图4-18.根据概率分布多次进行采样的例子.PNG)
图4-18.根据概率分布多次进行采样的例子

处理好高频单词才能获得更好的结果。

In [5]:
# 使用 Python 来说明基于概率分布的采样
import numpy as np

# 从 0-9 随机选择一个数字
np.random.choice(10)

9

In [7]:
# 从words列表中随机选择一个元素
words = ['you', 'say', 'goodbye', 'I', 'hello', '.']
np.random.choice(words)

'goodbye'

In [8]:
# 有放回采样5次
np.random.choice(words, size=5)

array(['goodbye', 'goodbye', 'I', 'goodbye', 'you'], dtype='<U7')

In [9]:
# 无放回采样5次
np.random.choice(words, size=5, replace=False)

array(['I', 'say', '.', 'hello', 'you'], dtype='<U7')

In [14]:
# 基于概率分布进行采样
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
np.random.choice(words, p=p)

'you'

$$
    P'(w_i)=\frac{P(w_i)^{0.75}}{\sum_j^nP(w_j)^{0.75}} \tag{4.4}
$$

对原来的概率分布的各个元素取 0.75 次方，防止低频单词被忽略。更准确地说，通过取 0.75 次方，低频单词的概率将稍微变高。

In [15]:
p = [0.7, 0.29, 0.01]
new_p = np.power(p, 0.75)
new_p /= np.sum(new_p)
print(new_p)

[0.64196878 0.33150408 0.02652714]


In [17]:
from negative_sampling_layer import *

corpus = np.array([0, 1, 2, 3, 4, 1, 2, 3])
power = 0.75
sample_size = 2

sampler = UnigramSampler(corpus, power, sample_size)
target = np.array([1, 3, 0])
negative_sample = sampler.get_negative_sampler(target)
print(negative_sample)

[[4 2]
 [2 4]
 [3 1]]


### 4.2.7 负采样的实现

实现负采样。我们把它实现为 NegativeSamplingLoss 类，（[ch04/negative_sampling_layer.py](../ch04/negative_sampling_layer.py)）。

## 4.3 改进版word2vec的学习

### 4.3.1 CBOW模型的实现

将上下文部分扩展为可以处理任意的窗口大小。（[ch04/cbow.py](../ch04/cbow.py)）。

![](../images/图4-19.用单词ID表示上下文和目标词的例子：这里显示的是窗口大小为1的上下文.PNG)
图4-19　用单词ID表示上下文和目标词的例子：这里显示的是窗口大小为1的上下文

### 4.3.2 CBOW模型的学习代码

（[ch04/train.py](../ch04/train.py)）

### 4.3.3 CBOW模型的评价

使用 word2vec 的单词的分布式表示，可以通过向量的加减法来解决类推问题。

![](../images/图4-20.展示各个单词在单词向量空间上的相关性.PNG)
图4-20.借助“man : woman = king : ?”这个类推问题，展示各个单词在单词向量空间上的相关性

使用 word2vec 的单词的分布式表示，可以通过向量的加减法来解决类推问题。

## 4.4 word2vec相关的其他话题

### 4.4.1 word2vec的应用例

在自然语言处理领域，单词的分布式表示之所以重要，原因就在于**迁移学习**（transfer learning）。
迁移学习是指在某个领域学到的知识可以被应用于其他领域。

单词的分布式表示的优点是可以将单词转化为固定长度的向量。另外，
使用单词的分布式表示，也可以将文档（单词序列）转化为固定长度的向量。

![](../images/图4-21.使用了单词的分布式表示的系统的处理流程.PNG)
图4-21.使用了单词的分布式表示的系统的处理流程

![](../images/图4-22.邮件的自动分类系统（情感分析）的例子.PNG)
图4-22.邮件的自动分类系统（情感分析）的例子


### 4.4.2 单词向量的评价方法

![](../images/图4-23.基于类推问题的单词向量的评价结果.PNG)
图4-23.基于类推问题的单词向量的评价结果

## 本章所学的内容

* Embedding 层保存单词的分布式表示，在正向传播时，提取单词 ID对应的向量
* 因为 word2vec 的计算量会随着词汇量的增加而成比例地增加，所以最好使用近似计算来加速
* 负采样技术采样若干负例，使用这一方法可以将多分类问题转化为二分类问题进行处理
* 基于 word2vec 获得的单词的分布式表示内嵌了单词含义，在相似的上下文中使用的单词在单词向量空间上处于相近的位置
* word2vec 的单词的分布式表示的一个特性是可以基于向量的加减法运算来求解类推问题
* word2vec 的迁移学习能力非常重要，它的单词的分布式表示可以应用于各种各样的自然语言处理任务